In [53]:
import pandas as pd
import re
import datetime

In [ ]:
df = pd.read_csv('..\\Project 2,3,4\\info_cars 1520_1999.csv',usecols=list(range(9))+[18])
df.head(10)

In [67]:
def split_name_and_price(s):
    if pd.isna(s):
        return pd.Series([None, None])
    s_str = str(s)
    if '-' in s_str:
        parts = s_str.rsplit('-', 1)
        if len(parts) == 2:
            return pd.Series([parts[0].strip(), parts[1].strip()])
    return pd.Series([s_str.strip(), None])

def normalize_price(price_str):
    """Chuyển đổi giá về đơn vị triệu VNĐ"""
    if pd.isna(price_str):
        return None
    price_str = str(price_str).strip()
    # Tìm tất cả các số trong chuỗi
    numbers = re.findall(r'\d+', price_str)
    if not numbers:
        return None
    
    # Chuyển đổi dựa trên đơn vị
    if 'Tỷ' in price_str or 'tỷ' in price_str:
        ty = int(numbers[0]) * 1000  # 1 tỷ = 1000 triệu
        trieu = int(numbers[1]) if len(numbers) > 1 else 0
        return ty + trieu
    elif 'Triệu' in price_str or 'triệu' in price_str:
        return int(numbers[0])
    else:
        return int(numbers[0])
    
def extract_daytime(s):
    if pd.isna(s):
        return None
    match = re.search(r'(\d{1,2})/(\d{1,2})/(\d{4})', s)
    if match:
        day, month, year = map(int, match.groups())
        date_obj = datetime.datetime(year,month,day)
        return str(date_obj.strftime(f"%d/%m/%Y"))
    return None

def parse_km(s):
    if pd.isna(s):
        return None
    if not isinstance(s, str):
        return None
    # Xóa mọi ký tự không phải số
    digits = re.sub(r'[^\d]', '', s)
    return int(digits) if digits else None

In [71]:
df = pd.read_csv('..\\Project 2,3,4\\info_cars 1520_1999.csv',usecols=list(range(9))+[18])
dataframe = df

# DEBUG: Xem dữ liệu cột đầu tiên
# print("=== 5 dòng đầu của cột 0 ===")
# print(dataframe.iloc[:5, 0])
# print("\n")

# Áp dụng hàm lên CỘT ĐẦU TIÊN (index 0)
dataframe[['name','price']] = dataframe.iloc[:, 0].apply(split_name_and_price)

# Áp dụng cho cột price
dataframe['price'] = dataframe['price'].apply(normalize_price)

# Lấy ngày ra khỏi cột sell_time
dataframe['sell_time'] = dataframe['sell_time'].apply(extract_daytime)

# Áp dụng lấy số km đã đi
dataframe['traveled_kilometer'] = dataframe['traveled_kilometer'].apply(parse_km)

# print(dataframe[['name','sell_time','price','traveled_kilometer']].head(50))
dataframe.to_csv('ouput.csv',index=False)